# Entrenamiento del modelo DialoGPT

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate -U
!pip install transformers[torch]
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (

You will need to setup git, adapt your email and name in the following cell.

In [2]:
!git config --global user.email "luisangel.motta@alumnos.viu.es"
!git config --global user.name "AngelMV97"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset
# Cargue del dataset
raw_datasets = load_dataset("LuangMV97/EmpathetiCounseling_Dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/47986 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12014 [00:00<?, ? examples/s]

In [5]:
for key in raw_datasets["train"][1304]:
    print(f"{key.upper()}: {raw_datasets['train'][1304][key]}")

INPUT: I feel like I am internally screaming all the time. Externally, I am calm, but I have this intense, unidentifiable emotion constantly. I don't feel sad. I don't cry a lot. I feel rather emotionally distant other than the internal screaming. It kind of feels like I'm a caged animal.
LABEL: It sounds like you are trying to find a label to help you to understand why you are feeling the way you are feeling.  Sometimes there are no labels for how we feel.  Sometimes there is no name for the emotions that we are experiencing. Regardless, you do not have to continue to feel this way, no matter what you call it.  If you are seeking to experience an internal calmness such as your external calmness, you may want to try some mindfulness practices or meditation.  However, being able to get at the root cause of what you are feeling may be what you need to help you release this feeling for the long term.  I recommend seeking support for this to allow this internal screaming to get some air an

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling

context_length = 512
checkpoint = "microsoft/DialoGPT-small"
#checkpoint = "LuangMV97/DialoGPT_EmpAI_final"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
def tokenize_function(example):
    return tokenizer(text=example["input"],
                     text_target=example["label"],
                     truncation=True,
                     padding='max_length',
                     max_length=context_length,
                     return_length=True)
# Tokenización del conjunto de datos
tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = raw_datasets.map(tokenize_function,
                                      batched=True,
                                      remove_columns=raw_datasets["train"].column_names)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Map:   0%|          | 0/47986 [00:00<?, ? examples/s]

Map:   0%|          | 0/12014 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'labels'],
        num_rows: 47986
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'labels'],
        num_rows: 12014
    })
})

In [9]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 512])
attention_mask shape: torch.Size([5, 512])
length shape: torch.Size([5])
labels shape: torch.Size([5, 512])


In [14]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="DialoGPT_EmpAI",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    auto_find_batch_size=True,      # Evita CUDA Out-of-Memory errors.
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    learning_rate=1e-5,
    weight_decay=5e-2,
    warmup_steps=16_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Pruebas de entrenamiento del modelo

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,2.685100,2.729595
2,2.567000,2.694819


Epoch,Training Loss,Validation Loss
0,2.685100,2.729595
2,2.567000,2.694819
4,2.439000,2.666931


TrainOutput(global_step=29990, training_loss=2.5825205366307, metrics={'train_runtime': 6603.1507, 'train_samples_per_second': 36.336, 'train_steps_per_second': 4.542, 'total_flos': 6.2687093981184e+16, 'train_loss': 2.5825205366307, 'epoch': 5.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 14.40


In [ ]:
eval_results

{'eval_loss': 2.666930675506592,
 'eval_runtime': 110.556,
 'eval_samples_per_second': 108.669,
 'eval_steps_per_second': 27.172,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712707715.8d5fc823c0cc.2331.1:   0%|          | 0.00/364 [00:00<?, ?B/s]

events.out.tfevents.1712700943.8d5fc823c0cc.2331.0:   0%|          | 0.00/19.2k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/DialoGPT_EmpAI_final_FineTuned2/commit/ac5d772827dc86e558888bdef59dbfa3a105cbb5', commit_message='End of training', commit_description='', oid='ac5d772827dc86e558888bdef59dbfa3a105cbb5', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,2.682600,2.727427
2,2.527600,2.681614
4,2.421900,2.667446


TrainOutput(global_step=29990, training_loss=2.5592963075908117, metrics={'train_runtime': 7778.8781, 'train_samples_per_second': 30.844, 'train_steps_per_second': 3.855, 'total_flos': 6.2687093981184e+16, 'train_loss': 2.5592963075908117, 'epoch': 5.0})

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.050400,2.982365
2,2.890400,2.897814
3,2.752700,2.810718
4,2.493900,2.736496
5,2.301200,2.709662


TrainOutput(global_step=3750, training_loss=2.6966758626302085, metrics={'train_runtime': 6776.0505, 'train_samples_per_second': 35.417, 'train_steps_per_second': 0.553, 'total_flos': 6.270747475968e+16, 'train_loss': 2.6966758626302085, 'epoch': 5.0})

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712624822.50e851074a66.712.0:   0%|          | 0.00/19.2k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/DialoGPT_EmpAI_final_FineTuned/commit/b6964dd5c42cdb9899bf68c9c273e8230fafa1a7', commit_message='End of training', commit_description='', oid='b6964dd5c42cdb9899bf68c9c273e8230fafa1a7', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=0.1, learning_rate=1e-5, warmup_sterps=5_000 DialoGPT_EmpAI3_FineTuned batch_size=32vs8

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,6.584800,4.042256
2,3.712700,3.385333
3,3.475800,3.201636
4,3.234200,3.097782
5,3.143900,3.024784


TrainOutput(global_step=3750, training_loss=3.9346263997395834, metrics={'train_runtime': 6702.2909, 'train_samples_per_second': 35.807, 'train_steps_per_second': 0.56, 'total_flos': 6.270747475968e+16, 'train_loss': 3.9346263997395834, 'epoch': 5.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 20.59


In [ ]:
eval_results

{'eval_loss': 3.0247838497161865,
 'eval_runtime': 228.4263,
 'eval_samples_per_second': 52.542,
 'eval_steps_per_second': 13.138,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712457114.a6947cfa41eb.1266.0:   0%|          | 0.00/7.98k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1712464149.a6947cfa41eb.1266.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/DialoGPT_EmpAI3_2/commit/a2947f7a74c781aa9eaecd675062cf89333d30ad', commit_message='End of training', commit_description='', oid='a2947f7a74c781aa9eaecd675062cf89333d30ad', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=5e-2, learning_rate=1e-4, warmup_sterps=4_000 DialoGPT_EmpAI3 batch_size=16

Epoch,Training Loss,Validation Loss
1,4.864400,3.137400
2,3.067500,2.918194
3,2.874500,2.790860
4,2.576700,2.685541


Epoch,Training Loss,Validation Loss
1,4.864400,3.137400
2,3.067500,2.918194
3,2.874500,2.790860
4,2.576700,2.685541
5,2.369900,2.626174


TrainOutput(global_step=3750, training_loss=3.0594512858072918, metrics={'train_runtime': 6065.564, 'train_samples_per_second': 39.566, 'train_steps_per_second': 0.618, 'total_flos': 6.270747475968e+16, 'train_loss': 3.0594512858072918, 'epoch': 5.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 13.82


In [ ]:
eval_results

{'eval_loss': 2.626174211502075,
 'eval_runtime': 96.2009,
 'eval_samples_per_second': 124.76,
 'eval_steps_per_second': 7.807,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712445873.7b0b5e4fbdc9.1259.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

events.out.tfevents.1712439596.7b0b5e4fbdc9.1259.0:   0%|          | 0.00/12.7k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/DialoGPT_EmpAI3/commit/1d378a0bd207b3f438f66b02db8d1ec95156458e', commit_message='End of training', commit_description='', oid='1d378a0bd207b3f438f66b02db8d1ec95156458e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=1e-2, learning_rate=1e-4, warmup_sterps=2_000 DialoGPT_EmpAI2

Epoch,Training Loss,Validation Loss
1,4.063800,3.010110


Epoch,Training Loss,Validation Loss
1,4.063800,3.010110
2,2.897600,2.805873
3,2.652400,2.656911
4,2.279900,2.600040
5,2.103900,2.608793


TrainOutput(global_step=3750, training_loss=2.749131705729167, metrics={'train_runtime': 6099.6004, 'train_samples_per_second': 39.345, 'train_steps_per_second': 0.615, 'total_flos': 6.443200217088e+16, 'train_loss': 2.749131705729167, 'epoch': 5.0})

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,5.012300,3.181849
2,3.119400,2.951047
3,2.935300,2.828770
4,2.661900,2.725851


Epoch,Training Loss,Validation Loss
1,5.012300,3.181849
2,3.119400,2.951047
3,2.935300,2.828770
4,2.661900,2.725851
5,2.475900,2.655762


TrainOutput(global_step=3750, training_loss=3.144549446614583, metrics={'train_runtime': 6073.8453, 'train_samples_per_second': 39.512, 'train_steps_per_second': 0.617, 'total_flos': 6.270747475968e+16, 'train_loss': 3.144549446614583, 'epoch': 5.0})

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.576600,3.905524
2,3.632600,3.355332
3,3.373800,3.177569


Epoch,Training Loss,Validation Loss
1,4.576600,3.905524
2,3.632600,3.355332
3,3.373800,3.177569
4,3.219300,3.078501
5,3.106700,3.009696


TrainOutput(global_step=7500, training_loss=3.8924258951822917, metrics={'train_runtime': 7969.5063, 'train_samples_per_second': 30.114, 'train_steps_per_second': 0.941, 'total_flos': 6.270747475968e+16, 'train_loss': 3.8924258951822917, 'epoch': 5.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 20.28


In [ ]:
eval_results

{'eval_loss': 3.0096957683563232,
 'eval_runtime': 243.1065,
 'eval_samples_per_second': 49.369,
 'eval_steps_per_second': 12.344,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712372192.06dfac5f27c8.533.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/DialoGPT_EmpAI_final2/commit/57f5c0272597949c1b49ef71ab8a356a4b050f50', commit_message='End of training', commit_description='', oid='57f5c0272597949c1b49ef71ab8a356a4b050f50', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Epoch,Training Loss,Validation Loss
6,3.028600,2.971438
7,2.961000,2.940672


Epoch,Training Loss,Validation Loss
6,3.028600,2.971438
7,2.961000,2.940672
8,2.892500,2.911088
9,2.829100,2.881528
10,2.761700,2.857724


TrainOutput(global_step=15000, training_loss=1.4531559977213542, metrics={'train_runtime': 7972.713, 'train_samples_per_second': 60.203, 'train_steps_per_second': 1.881, 'total_flos': 1.2541494951936e+17, 'train_loss': 1.4531559977213542, 'epoch': 10.0})

In [ ]:
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Epoch,Training Loss,Validation Loss
11,2.706100,2.812551


Epoch,Training Loss,Validation Loss
11,2.706100,2.812551
12,2.651500,2.798084
13,2.616500,2.782215
14,2.581300,2.768940
15,2.521300,2.754705


TrainOutput(global_step=22500, training_loss=0.8744909776475694, metrics={'train_runtime': 7471.7925, 'train_samples_per_second': 96.358, 'train_steps_per_second': 3.011, 'total_flos': 1.8812242427904e+17, 'train_loss': 0.8744909776475694, 'epoch': 15.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 15.72


In [ ]:
eval_results

{'eval_loss': 2.7547049522399902,
 'eval_runtime': 131.2189,
 'eval_samples_per_second': 91.465,
 'eval_steps_per_second': 11.439,
 'epoch': 15.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712380424.06dfac5f27c8.533.4:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

events.out.tfevents.1712389426.06dfac5f27c8.533.5:   0%|          | 0.00/364 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/DialoGPT_EmpAI_final2/commit/12842d3baed7445454578eedd5a84986b60a83cb', commit_message='End of training', commit_description='', oid='12842d3baed7445454578eedd5a84986b60a83cb', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.545900,3.293098


Epoch,Training Loss,Validation Loss
1,3.545900,3.293098
2,3.156500,3.042481
3,2.956200,2.932323
4,2.839100,2.880980
5,2.787200,2.868582


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=30000, training_loss=3.3142591064453124, metrics={'train_runtime': 7687.2417, 'train_samples_per_second': 31.219, 'train_steps_per_second': 3.903, 'total_flos': 6.270747475968e+16, 'train_loss': 3.3142591064453124, 'epoch': 5.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 17.61


In [ ]:
eval_results

{'eval_loss': 2.8685824871063232,
 'eval_runtime': 224.1031,
 'eval_samples_per_second': 53.556,
 'eval_steps_per_second': 13.391,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712344162.d16ded023349.4738.0:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1712352178.d16ded023349.4738.1:   0%|          | 0.00/364 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/DialoGPT_EmpAI_final/commit/ac02521892ebf567fadb76a86cc2e3cfaf8d7c6b', commit_message='End of training', commit_description='', oid='ac02521892ebf567fadb76a86cc2e3cfaf8d7c6b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=0, learning_rate=1e-4, warmup_sterps=1000 DialoGPT_EmpAI_smallPrueba3 1024

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
0,5.472600,3.187085
1,3.264900,2.956531
2,3.026300,2.875668
3,2.913100,2.831869
4,2.844500,2.801162
5,2.790200,2.784876
6,2.758000,2.773772
8,2.725900,2.767988


In [ ]:
trainer.train() #weight_decay=0.1, learning_rate=1e-5, warmup_sterps=1000 DialoGPT_EmpAI_smallPrueba2

Epoch,Training Loss,Validation Loss
0,No log,3.530258
2,3.453400,2.955134
4,3.005200,2.849108
6,2.862400,2.810997
8,2.807900,2.800506
9,2.800500,2.799879


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=4830, training_loss=3.205094653775233, metrics={'train_runtime': 7228.0734, 'train_samples_per_second': 42.801, 'train_steps_per_second': 0.668, 'total_flos': 8.0754131533824e+16, 'train_loss': 3.205094653775233, 'epoch': 9.99})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 16.44


In [ ]:
eval_results

{'eval_loss': 2.7998788356781006,
 'eval_runtime': 64.2916,
 'eval_samples_per_second': 120.327,
 'eval_steps_per_second': 15.041,
 'epoch': 9.99}

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1712181557.cdaaa872347b.3205.0:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

events.out.tfevents.1712188902.cdaaa872347b.3205.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/DialoGPT_EmpAI_smallPrueba2/commit/c26d15c6f700ff3ab2f7d4fe78a86b44df605c74', commit_message='End of training', commit_description='', oid='c26d15c6f700ff3ab2f7d4fe78a86b44df605c74', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Epoch,Training Loss,Validation Loss
7,2.894300,2.848246
9,2.892900,2.844596


Epoch,Training Loss,Validation Loss
7,2.894300,2.848246
9,2.882200,2.844204


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=4830, training_loss=0.8651740285427181, metrics={'train_runtime': 2182.4397, 'train_samples_per_second': 141.754, 'train_steps_per_second': 2.213, 'total_flos': 8.0755960578048e+16, 'train_loss': 0.8651740285427181, 'epoch': 10.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 17.19


In [ ]:
eval_results

{'eval_loss': 2.8442037105560303,
 'eval_runtime': 65.7935,
 'eval_samples_per_second': 117.58,
 'eval_steps_per_second': 14.697,
 'epoch': 10.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712181136.a1be0eee2fa5.731.5:   0%|          | 0.00/359 [00:00<?, ?B/s]

events.out.tfevents.1712178797.a1be0eee2fa5.731.4:   0%|          | 0.00/6.63k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/DialoGPT_EmpAI_small/commit/13289bbcde36802a56508aec16ee27ffa32ad910', commit_message='End of training', commit_description='', oid='13289bbcde36802a56508aec16ee27ffa32ad910', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,3.160470
2,3.206900,2.934499
4,3.013700,2.913240


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=2415, training_loss=3.34881195005176, metrics={'train_runtime': 3627.6978, 'train_samples_per_second': 42.64, 'train_steps_per_second': 0.666, 'total_flos': 4.0377980289024e+16, 'train_loss': 3.34881195005176, 'epoch': 4.99})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 18.42


In [ ]:
eval_results

{'eval_loss': 2.913240432739258,
 'eval_runtime': 65.2029,
 'eval_samples_per_second': 118.645,
 'eval_steps_per_second': 14.831,
 'epoch': 4.99}

In [ ]:
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
4,2.341500,2.640948


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=19335, training_loss=0.46674454765231044, metrics={'train_runtime': 5096.6811, 'train_samples_per_second': 30.35, 'train_steps_per_second': 3.794, 'total_flos': 2.806054637568e+17, 'train_loss': 0.46674454765231044, 'epoch': 5.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 13.98


In [ ]:
eval_results

{'eval_loss': 2.6378421783447266,
 'eval_runtime': 344.88,
 'eval_samples_per_second': 22.431,
 'eval_steps_per_second': 2.804,
 'epoch': 5.0}

In [ ]:
eval_results = trainer.evaluate()

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712172645.b3c2df730f2f.1113.3:   0%|          | 0.00/364 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/DialoGPT_EmpAI_DefPrueba2/commit/64a1fc86b8364b7f32fc892333e97ef9e6c1463b', commit_message='End of training', commit_description='', oid='64a1fc86b8364b7f32fc892333e97ef9e6c1463b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
0,3.032700,2.803351
1,2.672600,2.685382
2,2.478500,2.646108
3,2.368000,2.637650


Epoch,Training Loss,Validation Loss
0,3.032700,2.803351
1,2.672600,2.685382
2,2.478500,2.646108
3,2.368000,2.637650
4,2.339800,2.640813


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=19335, training_loss=2.671787451563308, metrics={'train_runtime': 25342.1605, 'train_samples_per_second': 6.104, 'train_steps_per_second': 0.763, 'total_flos': 2.80598208282624e+17, 'train_loss': 2.671787451563308, 'epoch': 5.0})

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

events.out.tfevents.1711985222.5e824229611a.719.1:   0%|          | 0.00/45.0k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/Roberta-DialoGPT_EmpAI_Prueba/commit/d0b36df901ec988341fbe388ee14685a87d711b9', commit_message='End of training', commit_description='', oid='d0b36df901ec988341fbe388ee14685a87d711b9', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=0, learning_rate=1e-5, 1024

Epoch,Training Loss,Validation Loss
0,0.115400,0.116726
1,0.103700,0.109285
2,0.097800,0.106526
3,0.095700,0.105459


Epoch,Training Loss,Validation Loss
0,0.115400,0.116726
1,0.103700,0.109285
2,0.097800,0.106526
3,0.095700,0.105459
4,0.094000,0.105388


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=11980, training_loss=0.1789425644532269, metrics={'train_runtime': 20917.1671, 'train_samples_per_second': 18.328, 'train_steps_per_second': 0.573, 'total_flos': 2.00331555766272e+17, 'train_loss': 0.1789425644532269, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.10538791120052338,
 'eval_runtime': 232.565,
 'eval_samples_per_second': 51.727,
 'eval_steps_per_second': 6.467,
 'epoch': 5.0}